In [1]:
using DataFrames, DataFramesMeta, Optim, Revise, Serialization, DebuggingUtilities, LinearAlgebra
using JuMP, Ipopt

In [2]:
string(@__DIR__) in LOAD_PATH || push!(LOAD_PATH, @__DIR__)
using BLPmodule; const m = BLPmodule;

In [3]:
ec = deserialize("jls/ec_nlls.jls");
n_firms = length(ec.firms);
q_obs = ec.q_obs;

---
### jump

In [206]:
t_indexer = [[] for _ in 1:length(ec.firms)]
for tt in eachindex(ec.tracts)
    for ind in ec.tracts[tt].inds
        push!(t_indexer[ind], tt)
    end
end

In [ ]:
nX = 1
nD = 1
model = Model(Ipopt.Optimizer)
θX = @variable(model, [ii=1:nX])
θD = @variable(model, [ii=1:nD])



In [285]:
function nlls(n_firms::Int, tracts::Vector, q_obs, t_indexer)

    model = Model(Ipopt.Optimizer)
    @variable(model, θ[1:2])


    n_tracts = length(tracts)
    nJ = [t.n_firms for t in tracts]
    M_vec = [tracts[t].M for t in eachindex(tracts)]
    X_vec = [[hcat(tracts[t].X[j, :], tracts[t].D[j, :]) for j in eachindex(tracts[t].firms)] for t in eachindex(tracts)]

    # function util_expr(model, theta, Xt, nJ)
    #     return @NLexpression(model, exp.([dot(Xt[jj,:], θ) for jj in 1:nJ]))
    # end

    # util_expr_vec = [util_expr(model, θ, X_vec[t], nJ[t]) for t in eachindex(tracts)]
    # denom_expr_vec = [@NLexpression(model, [j = 1:nJ[t]], 1+sum(util_expr_vec[t])) for t in eachindex(tracts)]
    # q_expr_vec = [@NLexpression(model, [j = 1:nJ[t]], util_expr_vec[t]./denom_expr_vec[t] .* M_vec[t]) for t in eachindex(tracts)]

    # u = @NLexpression(model, [jj=1:n_firms, tt=1:n_tracts], dot(Xt[jj,tt], θ) for jj in 1:nJ[tt])

    # u = @expression(model, [tt=1:n_tracts, jj = 1:nJ[tt]], X_vec[tt][jj] * θ)
    # @showln u[1,2]
    expu1 = @NLexpression(model, [tt=1:n_tracts, jj = 1:nJ[tt]], exp(X_vec[tt][jj][1] * θ[1]))


    q_mat = @variable(model, [t in 1:n_tracts, j in 1:n_firms])

    for t in 1:n_tracts
        q_mat[t, tracts[t].inds] = @NLexpression(model, [t = tracts[t].inds], 1.)
    end

    q_vec = @NLexpression(model, [j = 1:n_firms], sum(q_mat[j,which_t[j]]))
    @NLobjective(model, Min, reduce(+, (q_obs .- q_vec).^2))

    optimize!(model)

    return
end
    

nlls (generic function with 4 methods)

In [286]:
nlls(n_firms, ec.tracts, q_obs, t_indexer)

LoadError: MethodError: [0mCannot `convert` an object of type [92mNonlinearExpression[39m[0m to an object of type [91mVariableRef[39m
[0mClosest candidates are:
[0m  convert(::Type{T}, [91m::T[39m) where T at essentials.jl:205
[0m  VariableRef(::Any, [91m::Any[39m) at /users/facsupport/zhli/.julia/packages/JuMP/Z1pVn/src/variables.jl:203

In [93]:
model = Model(Ipopt.Optimizer)


A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Ipopt

In [111]:
@variable(model, u[i = 1:n_tracts, j = 1:n_firms])

LoadError: UndefVarError: n_tracts not defined

In [110]:
unregister(model, :u)


---

### optim

In [120]:
# closure 
function closure_nlls(θ2)
    ec = deserialize("jls/ec_nlls.jls");
    return m.nlls_obj(θ2, n_firms, ec.tracts, q_obs)
end;

In [132]:
init_θ = ones(4)

4-element Vector{Float64}:
 1.0
 1.0
 1.0
 1.0

In [133]:
q_pred = m.pred_q(init_θ, n_firms, ec.tracts);
q_pred[1:5]

[1.0][1.0, 1.0, 1.0]


5-element Vector{Float64}:
 1790.9103193676451
 1490.1837794631292
  214.6526910438062
  617.8090816559302
 1365.6500703525512

In [134]:
obj_init = m.nlls_obj(init_θ, n_firms, ec.tracts, q_obs)

[1.0][1.0, 1.0, 1.0]


4.0142834009239566e8

In [135]:
res = optimize(closure_nlls, init_θ, LBFGS())

[1.0000060554544523, 1.0][1.0, 1.0]
[0.9999939445455476, 1.0][1.0, 1.0]
[1.0, 1.0000060554544523][1.0, 1.0]
[1.0, 0.9999939445455476][1.0, 1.0]
[1.0, 1.0][1.0000060554544523, 1.0]
[1.0, 1.0][0.9999939445455476, 1.0]
[1.0, 1.0][1.0, 1.0000060554544523]
[1.0, 1.0][1.0, 0.9999939445455476]
[1.0, 1.0][1.0, 1.0]
[-2.5228189706905308e8, -7.616584151922531e7][4769.870075784511, -2997.3069616205266]
[-2.5228495245062855e8, -7.616584151922531e7][4769.870075784511, -2997.3069616205266]
[-2.5228342475984082e8, -7.616538030044116e7][4769.870075784511, -2997.3069616205266]
[-2.5228342475984082e8, -7.616630273800947e7][4769.870075784511, -2997.3069616205266]
[-2.5228342475984082e8, -7.616584151922531e7][4769.898959515499, -2997.3069616205266]
[-2.5228342475984082e8, -7.616584151922531e7][4769.841192053523, -2997.3069616205266]
[-2.5228342475984082e8, -7.616584151922531e7][4769.870075784511, -2997.2888115647406]
[-2.5228342475984082e8, -7.616584151922531e7][4769.870075784511, -2997.3251116763126]
[-2

 * Status: success

 * Candidate solution
    Final objective value:     5.306161e+06

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 2.52e+08 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.00e+00 ≰ 0.0e+00
    |f(x) - f(x')|         = 4.60e+08 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 8.68e+01 ≰ 0.0e+00
    |g(x)|                 = 0.00e+00 ≤ 1.0e-08

 * Work counters
    Seconds run:   1  (vs limit Inf)
    Iterations:    1
    f(x) calls:    2
    ∇f(x) calls:   2


In [136]:
min = Optim.minimizer(res)

4-element Vector{Float64}:
    -2.5228342475984082e8
    -7.616584151922531e7
  4769.870075784511
 -2997.3069616205266

In [137]:
q_pred = m.pred_q(min, n_firms, ec.tracts);

[-2.5228342475984082e8][-7.616584151922531e7, 4769.870075784511, -2997.3069616205266]


In [138]:
println(q_pred[1:30])
println(q_obs[1:30])

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[80.0, 81.0, 54.0, 55.0, 121.0, 147.0, 158.0, 99.0, 139.0, 45.0, 91.0, 147.0, 86.0, 135.0, 151.0, 122.0, 129.0, 105.0, 85.0, 92.0, 219.0, 225.0, 113.0, 91.0, 69.0, 44.0, 115.0, 87.0, 78.0, 114.0]
